In [1]:
import pandas as pd
import requests

In [289]:
API_KEY = '881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36'

In [291]:
next_url = "https://api.agendor.com.br/v3/deals"

# Cabeçalhos da requisição
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

params={"per_page": 100}

all_json_deals_data = []

while next_url:
    response = requests.get(next_url, headers=headers, params=params)
    json_deal_data = response.json()['data']
    all_json_deals_data.extend(json_deal_data)
    links = response.json()['links']
    if 'next' in links.keys():
        next_url = links['next']
    else:
        next_url = False




In [293]:
import json

# Salvando como JSON
with open("deals.json", "w", encoding="utf-8") as arquivo:
    json.dump(all_json_deals_data, arquivo, indent=4, ensure_ascii=False)

In [2]:
import json

# Ler arquivo JSON com vários objetos
with open("deals.json", "r", encoding="utf-8") as arquivo:
    deals = json.load(arquivo)

In [9]:
titles = {'title': []}
for deal in deals:
    titles['title'].append(deal['title'])

In [25]:
df = pd.DataFrame(titles)

In [39]:
deals[:30]

[{'id': 29068938,
  'accountId': 698720,
  'organization': None,
  'person': {'id': 55574570, 'name': 'Andre Dias De Matos', 'email': None},
  'author': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'dealStage': {'id': 3186643,
   'name': '2.1 VALIDAÇÃO',
   'sequence': 1,
   'funnel': {'id': 773196, 'name': '2 SERVICE'}},
  'dealStatus': {'id': 2, 'name': 'Ganho'},
  'lossReason': None,
  'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
  'title': 'Andre Dias de Matos - 02 ',
  'description': None,
  'startTime': '2025-01-24T13:33:30.000Z',
  'endTime': '2025-02-04T00:00:00.000Z',
  'wonAt': '2025-02-04T20:33:52.000Z',
  'lostAt': None,
  'createdAt': '2025-01-24T13:33:30.000Z',
  'updatedAt': '2025-02-04T20:33:52.000Z',
  'ranking': 0,
  'value': 0.0,
  'products': [],
  'allowedUsers': [{'accountId': 698720,
    'id': 860639,
    'name': 'Guilherme Hely'},
   {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'},
   {'accountId': 698720, '

In [85]:
pd.DataFrame(deal_by_stage).to_csv('deal_by_stage.csv', index=False, encoding='utf-8')

In [83]:
deal_by_stage = {
    'stage_detail': [], 'stage_name': [], 'stage_number': [], 'stage_status': [],
    'person': [], 'title': [], 'date_created': [], 'date_lost': [], 'date_won': [],
    'organization': []
}
for deal in deals:
    stage_detail = deal['dealStage']['name']
    stage_number = int(deal['dealStage']['sequence'])
    stage_name = deal['dealStage']['funnel']['name']
    stage_status = deal['dealStatus']['name']
    if deal['person']:
        person = deal['person']['id']
    else:
        person = None
    if deal['organization']:
        organization = deal['organization']['id']
    else:
        organization = None
    date_created = deal['createdAt']
    date_won = deal['wonAt']
    date_lost = deal['lostAt']
    title = deal['title']
    deal_by_stage['stage_detail'].append(stage_detail)
    deal_by_stage['stage_number'].append(stage_number)
    deal_by_stage['stage_name'].append(stage_name)
    deal_by_stage['stage_status'].append(stage_status)
    deal_by_stage['person'].append(person)
    deal_by_stage['title'].append(title)
    deal_by_stage['date_created'].append(date_created)
    deal_by_stage['date_lost'].append(date_lost)
    deal_by_stage['date_won'].append(date_won)
    deal_by_stage['organization'].append(organization)

    

In [71]:
for deal in deals:
    if deal['organization']:
        print(deal)

{'id': 28516943, 'accountId': 698720, 'organization': {'id': 41011943, 'name': 'Denilson', 'email': None}, 'person': None, 'author': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'}, 'dealStage': {'id': 3226395, 'name': '5.1 CONTATO WHATSAPP', 'sequence': 1, 'funnel': {'id': 773194, 'name': '5 SAQUE DIGITAL'}}, 'dealStatus': {'id': 3, 'name': 'Perdido'}, 'lossReason': {'id': 2898658, 'name': 'Sem contato', 'description': None}, 'owner': {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'}, 'title': 'Denilson ', 'description': None, 'startTime': '2024-12-18T17:44:44.000Z', 'endTime': '2025-01-16T00:00:00.000Z', 'wonAt': None, 'lostAt': '2025-01-16T14:11:02.000Z', 'createdAt': '2024-12-18T17:44:44.000Z', 'updatedAt': '2025-01-16T14:11:02.000Z', 'ranking': 0, 'value': 0.0, 'products': [], 'allowedUsers': [{'accountId': 698720, 'id': 860639, 'name': 'Guilherme Hely'}, {'accountId': 698720, 'id': 860979, 'name': 'Camila Neves'}, {'accountId': 698720, 'id': 860980, 'name': 